In [ ]:
import os,sys
from tqdm import tqdm
from scipy.sparse import csc_matrix, coo_matrix, csr_matrix, lil_matrix
import sys
import numpy as np
import random
import numpy.random as nr
import scipy.linalg as sla
from scipy.sparse import coo_matrix
!ls

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
from matplotlib import cm

nodes_u = set()
nodes_v = set()
weight_edges = list()
edgeDict=dict()

G = nx.Graph()
with open("yelp.edgelist.inject.1000", encoding='utf-8') as file:
    for line in file:
        l = line.split(' ')[0:3]
       
        nodes_u.add('u' + l[0])
        nodes_v.add('v' + l[1])

        weight_edges.append(['u' + l[0], 'v' + l[1], int(l[2])])

nodes = list(nodes_u) + list(nodes_v)
node2index = dict([(node, index) for index, node in enumerate(nodes)])

G = nx.Graph()
G.add_nodes_from([index for index in range(len(nodes_u))], bipartite=0)
G.add_nodes_from([index + len(nodes_u) for index in range(len(nodes_v))], bipartite=1)

for l in weight_edges:
    G.add_edge(node2index[l[0]], node2index[l[1]], weight = l[2])

In [ ]:
#result subgraph load
fraudar_user=[]
detect_user=[]
false_fraudar_user=[]
with open("yelp.edgelist.trueA.1000", encoding='utf-8') as file:
    for line in file:
        fraudar_user.append('u' +  line.strip())
                            
with open("yelp.blk1.rows", encoding='utf-8') as file:
    for line in file:
        detect_user.append('u' +  line.strip())

false_fraudar_user = list(set(detect_user).difference(set(fraudar_user)))
fraudar_item=[]
with open("yelp.edgelist.trueB.1000", encoding='utf-8') as file:
    for line in file:
        fraudar_item.append('v' +  line.strip())
sub_nodes = fraudar_user + fraudar_item

def subgraph(sub_nodes):
    sub_node_index = [node2index[node] for node in sub_nodes]

    node_color_dict={}
    for index in range(len(sub_nodes)):
        node = sub_nodes[index]
        color = 'blue'
        if node in  fraudar_user:
            color = 'red'
        if node in fraudar_item:
            color = 'yellow'
        node_color_dict[sub_node_index[index]] = color
    return sub_node_index, node_color_dict

In [ ]:
#plt.figure(figsize=(100,100), dpi=1)
plt.rcParams['figure.dpi'] = 30
plt.rcParams['figure.figsize'] = [100, 100]

sub_node_index, node_color_dict = subgraph(sub_nodes)
H = G.subgraph(sub_node_index)
color_map = [node_color_dict[node] for node in H.nodes]
pos=nx.spring_layout(H, iterations=100)
nx.draw(H, node_color = color_map, with_labels = False)
plt.show

In [ ]:
plt.rcParams['figure.dpi'] = 30
plt.rcParams['figure.figsize'] = [40, 40]
sm = nx.to_numpy_matrix(H)
plt.spy(sm, marker=',', markersize = 5)
plt.show()

In [ ]:
#plt.figure(figsize=(100,100), dpi=1)
plt.rcParams['figure.dpi'] = 30
plt.rcParams['figure.figsize'] = [100, 100]

sub_node_index, node_color_dict = subgraph(detect_user + fraudar_item)
H = G.subgraph(sub_node_index)
color_map = [node_color_dict[node] for node in H.nodes]
pos=nx.spring_layout(H, iterations=100)
nx.draw(H, node_color = color_map, with_labels = False)
plt.show

In [ ]:
plt.rcParams['figure.dpi'] = 30
plt.rcParams['figure.figsize'] = [40, 40]
sm = nx.to_numpy_matrix(H)
plt.spy(sm, marker=',', markersize = 5)
plt.show()

In [ ]:
#plt.figure(figsize=(100,100), dpi=1)
plt.rcParams['figure.dpi'] = 30
plt.rcParams['figure.figsize'] = [100, 100]

sub_node_index, node_color_dict = subgraph(false_fraudar_user + fraudar_item)
H = G.subgraph(sub_node_index)
color_map = [node_color_dict[node] for node in H.nodes]
pos=nx.spring_layout(H, iterations=100)
nx.draw(H, node_color = color_map, with_labels = False)
plt.show

In [ ]:
plt.rcParams['figure.dpi'] = 30
plt.rcParams['figure.figsize'] = [40, 40]
sm = nx.to_numpy_matrix(H)
plt.spy(sm, marker=',', markersize = 5)
plt.show()

In [ ]:
sub_node_index, node_color_dict = subgraph(sub_nodes)
H = G.subgraph(sub_node_index)
GM = nx.algorithms.isomorphism.GraphMatcher(G,H)
for subgraph in GM.subgraph_isomorphisms_iter():
    print(subgraph)